# Feinabstimmung von Open AI Modellen

Dieses Notebook basiert auf den aktuellen Empfehlungen in der [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) Dokumentation von Open AI.

Die Feinabstimmung verbessert die Leistung von Foundation-Modellen für Ihre Anwendung, indem das Modell mit zusätzlichen, für den jeweiligen Anwendungsfall oder das Szenario relevanten Daten und Kontexten nachtrainiert wird. Beachten Sie, dass Techniken wie _Few Shot Learning_ und _Retrieval Augmented Generation_ es ermöglichen, den Standard-Prompt mit relevanten Daten zu erweitern, um die Qualität zu steigern. Diese Ansätze sind jedoch durch die maximale Token-Fenstergröße des jeweiligen Foundation-Modells begrenzt.

Mit der Feinabstimmung trainieren wir das Modell selbst mit den benötigten Daten nach (wodurch wir viel mehr Beispiele verwenden können, als in das maximale Token-Fenster passen) – und stellen eine _individuelle_ Version des Modells bereit, die zur Inferenzzeit keine Beispiele mehr benötigt. Das verbessert nicht nur die Effektivität unseres Prompt-Designs (wir haben mehr Flexibilität, das Token-Fenster für andere Dinge zu nutzen), sondern kann auch unsere Kosten senken (da wir zur Inferenzzeit weniger Tokens an das Modell senden müssen).

Die Feinabstimmung besteht aus 4 Schritten:
1. Die Trainingsdaten vorbereiten und hochladen.
1. Den Trainingsjob ausführen, um ein feinabgestimmtes Modell zu erhalten.
1. Das feinabgestimmte Modell evaluieren und für Qualität iterieren.
1. Das feinabgestimmte Modell für die Inferenz bereitstellen, sobald Sie zufrieden sind.

Beachten Sie, dass nicht alle Foundation-Modelle die Feinabstimmung unterstützen – [prüfen Sie die OpenAI-Dokumentation](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) für aktuelle Informationen. Sie können auch ein bereits feinabgestimmtes Modell weiter feinabstimmen. In diesem Tutorial verwenden wir `gpt-35-turbo` als unser Ziel-Foundation-Modell für die Feinabstimmung.

---


### Schritt 1.1: Bereite dein Datenset vor

Wir bauen einen Chatbot, der dir hilft, das Periodensystem der Elemente zu verstehen, indem er Fragen zu einem Element mit einem Limerick beantwortet. In _diesem_ einfachen Tutorial erstellen wir nur ein Datenset, um das Modell mit einigen Beispielantworten zu trainieren, die das erwartete Format der Daten zeigen. In einem echten Anwendungsfall müsstest du ein Datenset mit deutlich mehr Beispielen erstellen. Möglicherweise kannst du auch ein offenes Datenset (für deinen Anwendungsbereich) verwenden, falls eines existiert, und es für das Fine-Tuning entsprechend umformatieren.

Da wir uns auf `gpt-35-turbo` konzentrieren und eine Einzelantwort (Chat Completion) erwarten, können wir Beispiele mit [diesem vorgeschlagenen Format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) erstellen, das die Anforderungen für OpenAI Chat Completions widerspiegelt. Wenn du mehrstufige Konversationen erwartest, solltest du das [Multi-Turn-Beispielformat](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) verwenden, das einen `weight`-Parameter enthält, um anzugeben, welche Nachrichten im Fine-Tuning-Prozess verwendet werden sollen (oder nicht).

Für unser Tutorial nutzen wir das einfachere Einzelantwort-Format. Die Daten liegen im [jsonl-Format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) vor, mit jeweils einem Datensatz pro Zeile, wobei jeder als JSON-Objekt formatiert ist. Der folgende Ausschnitt zeigt 2 Datensätze als Beispiel – siehe [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) für das vollständige Beispielset (10 Beispiele), das wir für unser Fine-Tuning-Tutorial verwenden. **Hinweis:** Jeder Datensatz _muss_ in einer einzigen Zeile definiert sein (nicht über mehrere Zeilen verteilt, wie es bei formatierten JSON-Dateien üblich ist)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

In einem echten Anwendungsfall benötigst du für gute Ergebnisse eine deutlich größere Beispielsammlung – hier gibt es einen Kompromiss zwischen der Qualität der Antworten und dem Zeit-/Kostenaufwand für das Fine-Tuning. Wir verwenden ein kleines Set, damit wir das Fine-Tuning schnell abschließen und den Prozess veranschaulichen können. Sieh dir [dieses Beispiel aus dem OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) für ein komplexeres Fine-Tuning-Tutorial an.


### Schritt 1.2 Laden Sie Ihr Datenset hoch

Laden Sie die Daten mit der Files API [wie hier beschrieben](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) hoch. Beachten Sie, dass Sie, um diesen Code auszuführen, zuvor folgende Schritte erledigt haben müssen:
 - Das `openai` Python-Paket installiert haben (stellen Sie sicher, dass Sie eine Version >=0.28.0 für die neuesten Funktionen verwenden)
 - Die Umgebungsvariable `OPENAI_API_KEY` mit Ihrem OpenAI API-Schlüssel gesetzt haben
Weitere Informationen finden Sie im [Setup-Leitfaden](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), der für den Kurs bereitgestellt wurde.

Führen Sie nun den Code aus, um aus Ihrer lokalen JSONL-Datei eine Datei zum Hochladen zu erstellen.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Schritt 2.1: Erstellen Sie den Fine-Tuning-Job mit dem SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Schritt 2.2: Überprüfe den Status des Jobs

Hier sind einige Dinge, die du mit der `client.fine_tuning.jobs` API machen kannst:
- `client.fine_tuning.jobs.list(limit=<n>)` – Zeigt die letzten n Fine-Tuning-Jobs an
- `client.fine_tuning.jobs.retrieve(<job_id>)` – Zeigt Details zu einem bestimmten Fine-Tuning-Job an
- `client.fine_tuning.jobs.cancel(<job_id>)` – Bricht einen Fine-Tuning-Job ab
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` – Zeigt bis zu n Ereignisse des Jobs an
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Der erste Schritt im Prozess ist _die Validierung der Trainingsdatei_, um sicherzustellen, dass die Daten im richtigen Format vorliegen.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Schritt 2.3: Ereignisse verfolgen, um den Fortschritt zu überwachen


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Schritt 2.4: Status im OpenAI-Dashboard anzeigen


Du kannst den Status auch einsehen, indem du die OpenAI-Website besuchst und im Bereich _Fine-tuning_ der Plattform nachschaust. Dort wird dir der Status des aktuellen Jobs angezeigt, und du kannst auch die Historie vorheriger Ausführungen verfolgen. In diesem Screenshot siehst du, dass die vorherige Ausführung fehlgeschlagen ist und der zweite Durchlauf erfolgreich war. Zur Erklärung: Dies geschah, weil beim ersten Durchlauf eine JSON-Datei mit falsch formatierten Einträgen verwendet wurde – nachdem das behoben war, wurde der zweite Durchlauf erfolgreich abgeschlossen und das Modell stand zur Nutzung bereit.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.de.png)


Du kannst die Statusmeldungen und Kennzahlen auch weiter unten im visuellen Dashboard sehen, wie hier gezeigt:

| Nachrichten | Kennzahlen |
|:---|:---|
| ![Nachrichten](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.de.png) |  ![Kennzahlen](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.de.png)|


### Schritt 3.1: ID abrufen & Feinabgestimmtes Modell im Code testen


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Schritt 3.2: Feinabgestimmtes Modell im Playground laden & testen

Du kannst das feinabgestimmte Modell jetzt auf zwei Arten testen. Zum einen kannst du den Playground besuchen und im Dropdown-Menü „Models“ dein neu feinabgestimmtes Modell aus den aufgelisteten Optionen auswählen. Die andere Möglichkeit ist, die Option „Playground“ im Fine-tuning-Panel zu nutzen (siehe Screenshot oben). Damit öffnet sich die folgende _Vergleichsansicht_, in der die Foundation- und die feinabgestimmte Modellversion nebeneinander angezeigt werden, um sie schnell zu vergleichen.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.de.png)

Trage einfach den Systemkontext ein, den du in deinen Trainingsdaten verwendet hast, und gib deine Testfrage ein. Du wirst sehen, dass auf beiden Seiten der identische Kontext und die gleiche Frage übernommen werden. Starte den Vergleich und du erkennst die Unterschiede in den Ausgaben der beiden Modelle. _Beachte, wie das feinabgestimmte Modell die Antwort im von dir vorgegebenen Format ausgibt, während das Foundation-Modell einfach dem System-Prompt folgt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.de.png)

Dir wird auffallen, dass der Vergleich auch die Token-Anzahl für jedes Modell sowie die benötigte Zeit für die Inferenz anzeigt. **Dieses spezielle Beispiel ist sehr einfach gehalten, um den Ablauf zu zeigen, spiegelt aber kein echtes Datenset oder Szenario wider**. Es kann sein, dass beide Beispiele die gleiche Token-Anzahl zeigen (da Systemkontext und Nutzer-Prompt identisch sind), das feinabgestimmte Modell aber mehr Zeit für die Inferenz benötigt (Custom-Modell).

In der Praxis wirst du kein so einfaches Beispiel verwenden, sondern mit echten Daten feinabstimmen (z. B. Produktkatalog für den Kundenservice), wobei die Qualität der Antworten viel deutlicher wird. In _diesem_ Zusammenhang ist es mit dem Foundation-Modell aufwendiger, eine vergleichbare Antwortqualität zu erreichen – dazu ist mehr individuelles Prompt-Engineering nötig, was den Token-Verbrauch und möglicherweise auch die Verarbeitungszeit für die Inferenz erhöht. _Um das auszuprobieren, schau dir die Fine-tuning-Beispiele im OpenAI Cookbook an._



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ausgangssprache gilt als maßgebliche Quelle. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
